In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# Laden der CSV
df = pd.read_csv("/home/dl4/Task2/Listen/Labelvorbereitung_all.csv", sep=",")
pd.set_option('display.max_rows', df.shape[0]+1)

# Setzen der Filter für die CSV
#df = df[df.Sprungkategorie == "Doppelspringer"]
df = df[df.Sprungkategorie == "Einzelspringer"]
df = df[df.Qualität == "gut"]
#df = df[df.ID < 164]

In [ ]:
#Bearbeiten der Spalte "Videoname"
df['Video_Namen'] = df['Video_Namen'].str.rstrip('mp4') 
df['Video_Namen'] = df['Video_Namen'].str.rstrip('.')

# Pixelgröße bestimmen
img_resize = 128

# Erstellung zweier Listen für die Frames und die Label
Frames = []
Label = []

# Video in Frames umwandeln
for i in df.index:
    vidcap = cv2.VideoCapture('/home/dl4/Task2/Videos/'+df['Video_Namen'][i]+'.mp4')        
    success, image= vidcap.read()
    count = 1

    #Bilder anhand des eintages "First_Water_Frame" in "Airframes" (0) und "Waterframe" (1) labeln
    while success:
        if count < df['First_Water_Frame'][i]:
            
            # Filterauswahl für die Bilder
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
            
            # Gegebenenfalls Bildgröße anpassen
            if img_resize > 1:
                resized = cv2.resize(image, (img_resize, img_resize))
                Frames.append(resized)
            else:
                Frames.append(image)
            Label.append(0)
            success, image = vidcap.read()
            
            
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
            if img_resize > 1:
                resized = cv2.resize(image, (img_resize, img_resize))
                Frames.append(resized)
            else:
                Frames.append(image)
            Label.append(1)
            success, image = vidcap.read()

        count+=1 
        
# Die Listen Frames und Label werden in ein C gespeichert   
Allframes = {"frames": Frames, "label": Label}

In [ ]:
# Dictionary wird in einem Pickle gespeichert
import pickle 

with open('All_frames_RGB_128.pkl', 'wb') as f:
    pickle.dump(Allframes, f)